In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

4 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = None
train_size  = 45000
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
batch_size = 256
epochs = 200

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [6]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_valid = x_train_all[train_size:]
y_valid = y_train_all[train_size:]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [7]:
x_train, x_valid, x_test = x_train.reshape((-1, *image_shape)), x_valid.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(
    lambda image, label: (tf.image.random_flip_left_right(image), label)
).map(
    lambda image, label: (tf.image.resize_with_crop_or_pad(image, image_shape[0]+6, image_shape[1]+6), label)
).map(
    lambda image, label: (tf.image.random_crop(image, (image_shape[0], image_shape[1], image_shape[2])), label)
).map(
    lambda image, label: (tf.image.random_contrast(image, lower=0.2, upper=1.0), label)
).shuffle(
    100000
).batch(
    batch_size
).prefetch(10)

In [9]:
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(batch_size)

In [10]:
layers = tf.keras.layers

In [16]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(img_input)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
out = layers.Dense(10, kernel_initializer=tf.keras.initializers.GlorotNormal())(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [17]:
def scheduler(epoch, lr):
    if epoch < 10:
        return 1e-3
    elif epoch < 90:
        return 1e-2
    elif epoch < 150:
        return 1e-3
    else:
        return 1e-4

In [18]:
model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [19]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [15]:
model.fit(x=train_ds, validation_data=valid_ds, epochs=epochs, callbacks=[callback])

Epoch 1/200
176/176 [==============================] - 31s 175ms/step - loss: 2.1439 - accuracy: 0.2074 - val_loss: 1.9726 - val_accuracy: 0.3020
Epoch 2/200
176/176 [==============================] - 22s 125ms/step - loss: 1.9715 - accuracy: 0.2893 - val_loss: 1.8535 - val_accuracy: 0.3436
Epoch 3/200
176/176 [==============================] - 26s 145ms/step - loss: 1.8897 - accuracy: 0.3283 - val_loss: 1.7868 - val_accuracy: 0.3734
Epoch 4/200
176/176 [==============================] - 20s 116ms/step - loss: 1.8374 - accuracy: 0.3470 - val_loss: 1.7390 - val_accuracy: 0.3916
Epoch 5/200
176/176 [==============================] - 20s 116ms/step - loss: 1.8009 - accuracy: 0.3617 - val_loss: 1.7089 - val_accuracy: 0.3938
Epoch 6/200
176/176 [==============================] - 20s 116ms/step - loss: 1.7754 - accuracy: 0.3709 - val_loss: 1.7194 - val_accuracy: 0.3956
Epoch 7/200
176/176 [==============================] - 20s 116ms/step - loss: 1.7564 - accuracy: 0.3784 - val_loss: 1.6538 -

Epoch 57/200
176/176 [==============================] - 19s 107ms/step - loss: 0.3874 - accuracy: 0.8638 - val_loss: 0.7565 - val_accuracy: 0.7870
Epoch 58/200
176/176 [==============================] - 19s 108ms/step - loss: 0.3766 - accuracy: 0.8691 - val_loss: 0.8002 - val_accuracy: 0.7796
Epoch 59/200
176/176 [==============================] - 19s 108ms/step - loss: 0.3737 - accuracy: 0.8717 - val_loss: 0.8018 - val_accuracy: 0.7766
Epoch 60/200
176/176 [==============================] - 19s 108ms/step - loss: 0.3623 - accuracy: 0.8736 - val_loss: 0.8181 - val_accuracy: 0.7808
Epoch 61/200
176/176 [==============================] - 19s 108ms/step - loss: 0.3582 - accuracy: 0.8744 - val_loss: 0.8450 - val_accuracy: 0.7764
Epoch 62/200
176/176 [==============================] - 19s 108ms/step - loss: 0.3469 - accuracy: 0.8795 - val_loss: 0.8240 - val_accuracy: 0.7814
Epoch 63/200
176/176 [==============================] - 19s 108ms/step - loss: 0.3315 - accuracy: 0.8849 - val_loss: 0

176/176 [==============================] - 19s 108ms/step - loss: 0.0471 - accuracy: 0.9854 - val_loss: 1.1945 - val_accuracy: 0.8096
Epoch 168/200
176/176 [==============================] - 19s 108ms/step - loss: 0.0478 - accuracy: 0.9847 - val_loss: 1.1953 - val_accuracy: 0.8100
Epoch 169/200
176/176 [==============================] - 19s 108ms/step - loss: 0.0485 - accuracy: 0.9855 - val_loss: 1.1947 - val_accuracy: 0.8098
Epoch 170/200
176/176 [==============================] - 19s 108ms/step - loss: 0.0498 - accuracy: 0.9853 - val_loss: 1.1968 - val_accuracy: 0.8098
Epoch 171/200
176/176 [==============================] - 19s 107ms/step - loss: 0.0462 - accuracy: 0.9857 - val_loss: 1.1950 - val_accuracy: 0.8110
Epoch 172/200
176/176 [==============================] - 19s 107ms/step - loss: 0.0468 - accuracy: 0.9857 - val_loss: 1.1980 - val_accuracy: 0.8102
Epoch 173/200
176/176 [==============================] - 19s 107ms/step - loss: 0.0466 - accuracy: 0.9857 - val_loss: 1.1951 -

In [20]:
model.load_weights('./model_weights/simple_cnn_train=all_ce.h5')

In [18]:
model.evaluate(x=x_test, y=y_test_all, verbose=0)

[1.2271720170974731, 0.8075000643730164]

In [ ]:
model.save('./model_weights/simple_cnn_train=all_ce.h5')

In [19]:
tmp = onp.load('./npy/cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')
print("==========NTK============")
model.evaluate(tmp, y_test[:128], verbose=0)

==========NTK============


[1.7618050575256348, 0.6875]

In [20]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-ce.npy')
print("==========CE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========CE============


[1.594173789024353, 0.73583984375]

In [21]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-mse.npy')
print("==========MSE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========MSE============


[1.59250009059906, 0.71728515625]

In [27]:
tmp = onp.load('./npy/cifar-fgsm-eps-0.03-time-None-nngp.npy')
model.evaluate(tmp, y_test[:128], verbose=0)

[1.3792508840560913, 0.71875]

In [19]:
tmp = onp.load('./batch_NTK_simple.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[2.647125005722046, 0.65478515625]

In [20]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy')
print("==========small============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========small============


[3.4256112575531006, 0.58740234375]

In [21]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy')
print("==========small============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========small============


[1.802112340927124, 0.720703125]

In [21]:
tmp = onp.load('./batch_NTK_cnn_19.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[1.8956958055496216, 0.720703125]